In [1]:
from fn import op, _, F
from operator import add, mul
from itertools import repeat, chain
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal, getcontext, ROUND_DOWN
getcontext().prec=4
getcontext().rounding = ROUND_DOWN

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

def flatmap(f, items):
        return list(map(f, items))
    
def last_index(l):
    return len(l)-1

def retrieve_state(l, offset):
    return l[last_index(l) + offset + 1]
# Stochastic Process
# Input RNG Seed
def bound_norm_random(rng, low, high):
    # Add RNG Seed
    res = rng.normal((high+low)/2,(high-low)/6)
    if (res<low or res>high):
        res = bound_norm_random(rng, low, high)
    return Decimal(res)

def env_proc(trigger_step, update_f):
    def env_step_trigger(trigger_step, update_f, step):
        if step == trigger_step:
            return update_f
        else:
            return lambda x: x
    return partial(env_step_trigger, trigger_step, update_f)

def env_proc(trigger_step, update_f):
    def env_step_trigger(trigger_step, update_f, step):
        if step == trigger_step:
            return update_f
        else:
            return lambda x: x
    return partial(env_step_trigger, trigger_step, update_f)

def time_step(s, dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    if s['mech_step'] == 0:
        dt = datetime.strptime(dt_str, fromat_str)
        t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
        return t.strftime(fromat_str)
    else: 
        return dt_str

In [3]:
# def pipeline(*steps):
#     return reduce(lambda x, y: y(x), list(steps))
# def compose(*funcs):
#     return lambda x: reduce(lambda f, g: g(f), list(funcs), x)

In [5]:
seed = {
    'z': np.random.RandomState(1),
    'a': np.random.RandomState(2),
    'b': np.random.RandomState(3),
    'c': np.random.RandomState(3)
}


In [6]:
a = np.random.RandomState(1)
proc_one_coef_B = 1.3
b = np.random.RandomState(2)

def pipe_f(x):
    return x

In [7]:
# UI Behaviors per Mechanism
def b1m1(step, sL, s):
    return s['s1']
def b2m1(step, sL, s):
    return s['s1'] * s['s2']

def b1m2(step, sL, s):
    return s['s1']
def b2m2(step, sL, s):
    return s['s1'] / s['s2']

def b1m3(step, sL, s):
    return s['s1'] + seed['z'].randint(1000)
def b2m3(step, sL, s):
    ps = retrieve_state(sL, -3)
    return ps['s2']

# UI Internal States per Mechanism
def s1m1(step, sL, s, _input):
#     print(s['s1'])
    s['s1'] = s['s1']**2 + _input
def s2m1(step, sL, s, _input):
    s['s2'] = s['s2'] + 1 + _input

def s1m2(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(step, sL, s, _input):
    s['s2'] = s['s2']
    
def s1m3(step, sL, s, _input):
    s['s1'] = s['s1']
def s2m3(step, sL, s, _input):
    s['s2'] = s['s2'] + _input

# UI Exogenous States //per Mechanism 

proc_one_coef_A = 0.7
proc_one_coef_B = 1.3
def es3p1(step, sL, s, _input):
    s['s3'] = s['s3'] * bound_norm_random(seed['a'], proc_one_coef_A, proc_one_coef_B)
def es4p2(step, sL, s, _input):
    s['s4'] = s['s4'] * bound_norm_random(seed['b'], proc_one_coef_A, proc_one_coef_B)  
def es5p2(step, sL, s, _input):
    s['timestamp'] = time_step(s, s['timestamp'], seconds=1)


#add env process f(s) read from time es
# funcs execute in order
def env_a(x): 
    return x + 1
def env_b(x): 
    return x + 2
    
def what_ever(x): 
    return x + 1

# Genesis States    
state_dict = {
    's1': Decimal(2.0),
    's2': Decimal(4.0),
    's3': Decimal(0.0),
    's4': Decimal(0.0),
    'timestamp': '2018-10-01 15:16:24'
}

exogenous_states = {
    "s3": es3p1,
    "s4": es4p2,
    "timestamp": es5p2
}

# update time at the end of each pipeline (once per 'block')
env_processes = {
    "s3": env_proc(1, env_a),
    "s4": env_proc(1, pipe | env_b | what_ever)
}
    
mechanisms = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
        }
    },
    "m3": {
        "behaviors": {
            "b1": b1m3,
            "b2": b2m3
        },
        "states": {
            "s1": s1m3,
            "s2": s2m3,
        }
    }
}

In [8]:
# if beh list empty, repeat 0 x n_states in list 
def generate_config(mechanisms, exogenous_states):
    es_funcs = [exogenous_states[state] for state in list(exogenous_states.keys())]
    config = list(
        map(
            lambda m: (
                list(mechanisms[m]["states"].values()) + es_funcs,
                list(mechanisms[m]["behaviors"].values())
            ), 
            list(mechanisms.keys())
        )
    )
    return config

# partials = list(map(lambda f: partial(f, step, sL), funcs))
def getColResults(step, sL, s, funcs):
    return list(map(lambda f: f(step, sL, s), funcs))

def getBehaviorInput(step, sL, s, funcs): 
    return op.foldr(_ + _)(getColResults(step, sL, s, funcs))

def apply_env_proc(env_processes, state_dict, step):
    for state in state_dict.keys(): 
        if state in list(env_processes.keys()):
            state_dict[state] = env_processes[state](step)(state_dict[state])
#     return state_dict
            
def mech_step(m_step, sL, state_funcs, behavior_funcs, env_processes, t_step):
    # Purge Memory accociated with certain objs
    # Truncate reffs
    in_copy, mutatable_copy, out_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    # * force eval of _input withing state functions
    _input = getBehaviorInput(m_step, sL, last_in_obj, behavior_funcs)

    apply_env_proc(env_processes, last_mut_obj, t_step)
    
    for f in state_funcs:
        f(m_step, sL, last_mut_obj, _input)
        
    last_mut_obj["mech_step"], last_mut_obj["time_step"] = m_step, t_step
    out_copy.append(last_mut_obj)
    
    del last_in_obj, last_mut_obj, in_copy, mutatable_copy,
    return out_copy

def block_gen(states_list, configs, env_processes, t_step):
    m_step = 0
    states_list_copy = deepcopy(states_list)
    genesis_states = states_list_copy[-1]
    genesis_states['mech_step'], genesis_states['time_step'] = m_step, t_step
    states_list = [genesis_states]
    
    m_step += 1
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(m_step, states_list, s_conf, b_conf, env_processes, t_step)     
        m_step += 1
        
    t_step += 1    
    
    return states_list

def pipeline(states_list, configs, env_processes, time_seq):
    time_seq = [x + 1 for x in time_seq]
    simulation_list = [states_list]
    for time_step in time_seq:
        pipeline_run = block_gen(simulation_list[-1], configs, env_processes, time_step)
        head, *pipeline_run = pipeline_run
        simulation_list.append(pipeline_run)
    
    # should only occur once for genesis state, placed before loop
    head, *tail = simulation_list
    head[-1]['mech_step'], head[-1]['time_step'] = 0, 0
    simulation_list = head, *tail
    return simulation_list

In [9]:
states_list = [state_dict]
states_list
{'s1': 8.742503764992283e+170,
   's2': 1.119720780594405e+171,
   's3': 4.251811100418906,
   's4': 6.92728042985628,
   'timestamp': '2018-10-01 15:16:31',
   'mech_step': 3,
   'time_step': 7}
{'s1': Decimal('2.221384236472411425810660220E+330'),
   's2': Decimal('2.845096899831187764579913612E+330'),
   's3': Decimal('3.622519693611211445932083740'),
   's4': Decimal('5.161487081026327858663750787'),
   'timestamp': '2018-10-01 15:16:32',
   'mech_step': 3,
   'time_step': 8}

{'s1': Decimal('2.221384236472411425810660220E+330'),
 's2': Decimal('2.845096899831187764579913612E+330'),
 's3': Decimal('3.622519693611211445932083740'),
 's4': Decimal('5.161487081026327858663750787'),
 'timestamp': '2018-10-01 15:16:32',
 'mech_step': 3,
 'time_step': 8}

In [19]:
configs = generate_config(mechanisms, exogenous_states)

pipeline(states_list, configs, env_processes, range(19))
# a = block_gen(states_list, configs, env_processes, 1)
# b = block_gen(a, configs, env_processes, 2)

([{'s1': Decimal('2'),
   's2': Decimal('4'),
   's3': Decimal('0'),
   's4': Decimal('0'),
   'timestamp': '2018-10-01 15:16:24',
   'mech_step': 0,
   'time_step': 0}],
 [{'s1': Decimal('14'),
   's2': Decimal('15'),
   's3': Decimal('1.150'),
   's4': Decimal('3.045'),
   'timestamp': '2018-10-01 15:16:25',
   'mech_step': 1,
   'time_step': 1},
  {'s1': Decimal('28.93'),
   's2': Decimal('15'),
   's3': Decimal('2.336'),
   's4': Decimal('6.033'),
   'timestamp': '2018-10-01 15:16:25',
   'mech_step': 2,
   'time_step': 1},
  {'s1': Decimal('28.93'),
   's2': Decimal('544.9'),
   's3': Decimal('2.973'),
   's4': Decimal('9.469'),
   'timestamp': '2018-10-01 15:16:25',
   'mech_step': 3,
   'time_step': 1}],
 [{'s1': Decimal('1.661E+4'),
   's2': Decimal('1.632E+4'),
   's3': Decimal('3.170'),
   's4': Decimal('10.19'),
   'timestamp': '2018-10-01 15:16:26',
   'mech_step': 1,
   'time_step': 2},
  {'s1': Decimal('3.322E+4'),
   's2': Decimal('1.632E+4'),
   's3': Decimal('3.402'),


In [11]:
from decimal import Decimal, MAX_EMAX
# Decimal(2.5347751615574597e+167)**2
MAX_EMAX**5

999999999999999995000000000000000009999999999999999990000000000000000004999999999999999999

In [12]:
1000 * 1000

1000000

In [13]:
from datetime import datetime, timedelta
# from datetime import timedelta  
# st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
st = datetime.now()
print(st)
x = st + timedelta(seconds=60)
print(x)

2018-10-02 12:37:06.309267
2018-10-02 12:38:06.309267


In [14]:
s = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
datetime_object = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
datetime_object

datetime.datetime(2018, 10, 2, 12, 37, 6)

In [15]:
datetime.fromtimestamp()

TypeError: Required argument 'timestamp' (pos 1) not found

In [ ]:
from datetime import datetime, timedelta

# def time_step_func(year, month, day, hour, minute, second):
a = datetime(2016, 11, 15, 9, 59, 25)#.strftime("%Y-%m-%d %H:%M:%S")
b = a + timedelta(seconds=60) # days, seconds, then other fields.
print(a)
print(b)
st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
def time_step_func(dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    dt = datetime.strptime(dt_str, fromat_str)
    t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
    return t.strftime(fromat_str)

time_step_func('2018-10-01 15:16:24')

In [ ]:
def time_step_func(x, y):
    return x + y

In [ ]:
for m in M: #M is the parameter sweep array
    # for Pairwise, run below separately
    for n in N: #N is the total number of Montecarlo runs
        for t in T: #T[len(T)]-T[0] is the duration of each simulation
            # Pipeline
            for b in Behaviors:
                for M in Mechanisms:
                    for s in States:
                        update(s)

In [ ]:
[{'s1': 2}] + [{'s1': 3}]
list(zip([block_gen] * 2, range(2)))
len([0,1])

In [ ]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
def step_func(x, y):
    return x + y
pipeline(states_list, configs, env_processes, step_func)

In [ ]:
# def s3step1(step, sL, s, _input):
#     s['s3'] = s['s3']    
# # def s3step2(step, sL, s, _input):
#     s['s3'] = s['s3'] + 1
# def s3step3(step, sL, s, _input):
#     s['s3'] = s['s3'] * bound_norm_random(proc_one_coef_A, proc_one_coef_B)

In [ ]:
# 15 + 28.933333333333334 + 4

In [ ]:
# def multiply(x,y,z):
#         return x * y * z

# # create a new function that multiplies by 2
# dbl = partial(partial(multiply,2), multiply, 2)
# print(dbl(2))